In [38]:
!python inference_code.py

1/6 File Name: AN_ID_20210526104509_1.dcm
2/6 File Name: AN_ID_20210526112758_1.dcm
3/6 File Name: DCM_CHEST_00104.dcm
4/6 File Name: DCM_CHEST_00118.dcm
5/6 File Name: DCM_CHEST_00123.dcm
6/6 File Name: JB0006_CXR_0base_201229.dcm


C:\Users\mip_aic\anaconda3\lib\site-packages\torch\nn\functional.py:3060: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "
C:\Users\mip_aic\anaconda3\lib\site-packages\torch\nn\functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
C:\Users\mip_aic\anaconda3\lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (22

In [20]:
import os
import glob
import pydicom
import torch
from eff_unet_shuffle import EfficientUNet
from torch.nn import functional as F
import numpy as np

def adjust_size(image, N):

    _, _, H, W = image.shape

    pad_h = (N - H % N) if H % N != 0 else 0
    pad_w = (N - W % N) if W % N != 0 else 0

    pad_left = pad_right = pad_top = pad_bottom = 0

    if pad_h > 0:
        pad_bottom = pad_h
    if pad_w > 0:
        pad_right = pad_w

    padding = (pad_left, pad_right, pad_top, pad_bottom)

    if pad_right > 0 or pad_bottom > 0:
        image = F.pad(image, padding, mode='constant', value=0)

    return image, padding

def remove_padding(image, padding):
    pad_left, pad_right, pad_top, pad_bottom = padding
    # no padding return original
    if pad_right == 0 and pad_bottom == 0:
        return image

    _, _, H_padded, W_padded = image.shape

    H_original = H_padded - pad_bottom
    W_original = W_padded - pad_right
    return image[:, :, :H_original, :W_original]


def ddim_sample(model, condition, device='cuda'):
    # Time Step
    num_timesteps = 1000
    sample_step = 30
    
    new_timesteps = torch.linspace(num_timesteps - 1, 0, steps=sample_step, device=device).long()

    # Diffusion Params
    betas = torch.linspace(0.0001, 0.02, num_timesteps).to(device)
    alphas = 1.0 - betas
    alpha_cumprod = torch.cumprod(alphas, dim=0)

    x = torch.randn_like(condition)

    for step, i in enumerate(new_timesteps):
        t = torch.tensor([i], device=device, dtype=torch.long)

        with torch.no_grad():
            eps_theta = model(x, condition, t)

        alpha = alpha_cumprod[i]
        sqrt_alpha = torch.sqrt(alpha)
        sqrt_one_minus_alpha = torch.sqrt(1 - alpha)
        x0_pred = (x - sqrt_one_minus_alpha * eps_theta) / sqrt_alpha

        if step < sample_step - 1:
            next_i = new_timesteps[step + 1]
            alpha_next = alpha_cumprod[next_i]

            coef1 = torch.sqrt(alpha_next) * x0_pred
            coef2 = torch.sqrt(1 - alpha_next) * eps_theta
            x = coef1 + coef2
        else:
            x = x0_pred

    return x

In [21]:
weight_dir = "./bone_supp_diff.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(weight_dir, map_location=device)

In [3]:
model = EfficientUNet(in_channels=2, out_channels=1, init_features=64).to(device)
model.load_state_dict(checkpoint)
model.eval()

EfficientUNet(
  (encoder1): EfficientUNetBlock(
    (conv1): DepthwiseSeparableConv(
      (depthwise): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
      (pointwise): Conv2d(2, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (conv2): DepthwiseSeparableConv(
      (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
      (pointwise): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): GELU()
    (residual): Conv2d(2, 64, kernel_size=(1, 1), stride=(1, 1))
    (time_mlp1): Linear(in_features=64, out_features=2, bias=True)
    (time_mlp2): Linear(in_features=2, out_features=64, bias=True)
  )
  (encoder2): EfficientUNetBlock(
    (conv1): DepthwiseSeparableConv(
      (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
      (pointwise): Conv2d(64, 128, kernel_size=(1, 1)

In [48]:
scripted_model = torch.jit.script(model)
scripted_model.save("bone_supp_diff.mipx")  # 필요하면 저장

In [22]:
scripted_model = torch.jit.load("bone_supp_diff.mipx")

In [23]:
dcm_path = "./sample/DCM_CHEST_00118.dcm"

In [24]:
if not os.path.exists(dcm_path):
    print(f"File does not exist or is not readable: {dcm_path}")

In [25]:
ds = pydicom.read_file(dcm_path)

In [26]:
dcm_arr = np.array(ds.pixel_array, dtype='float32')

In [27]:
Q3 = np.percentile(dcm_arr, 75)
Q1 = np.percentile(dcm_arr, 25)
IQR = Q3-Q1
upper_boundary = Q3 + 1.5 *IQR
dcm_clip = np.clip(dcm_arr, 0, upper_boundary)
outlier_residual = dcm_arr - dcm_clip
# Normalization
max_val = np.max(dcm_clip)
dcm_max_norm = dcm_clip / max_val
mean_val = np.mean(dcm_max_norm)
std_val = np.std(dcm_max_norm)
dcm_norm = (dcm_max_norm - mean_val) / std_val
# to tensor
dcm_torch = torch.from_numpy(dcm_norm).unsqueeze(0).unsqueeze(0)
dcm_torch = dcm_torch.to(device).float()
# check downsampling level
b, c, h, w = dcm_torch.shape
max_size = np.maximum(h, w)
ds_factor = int(np.round(max_size / 512))
# padding and interpolation
dcm_torch, padding = adjust_size(dcm_torch, ds_factor)
dcm_torch = F.interpolate(dcm_torch, scale_factor=1/ds_factor, mode='bilinear')

In [28]:
#ddim_sample
condition = dcm_torch

In [29]:
num_timesteps = 1000
sample_step = 30
new_timesteps = torch.linspace(num_timesteps - 1, 0, steps=sample_step, device=device).long()

In [30]:
betas = torch.linspace(0.0001, 0.02, num_timesteps).to(device)
alphas = 1.0 - betas
alpha_cumprod = torch.cumprod(alphas, dim=0)

In [31]:
x = torch.randn_like(condition)
print("x")
print(x.mean())
print(x.max())
print(x.min())

x
tensor(-0.0010, device='cuda:0')
tensor(4.5144, device='cuda:0')
tensor(-4.9041, device='cuda:0')


In [32]:
for step, i in enumerate(new_timesteps):
    t = torch.tensor([i], device=device, dtype=torch.long)

    #print(x.shape)
    #print(condition.shape)
    #print(t.shape)

    with torch.no_grad():
        eps_theta = scripted_model(x, condition, t)
        #eps_theta = model(x, condition, t)

    alpha = alpha_cumprod[i]
    sqrt_alpha = torch.sqrt(alpha)
    sqrt_one_minus_alpha = torch.sqrt(1 - alpha)
    x0_pred = (x - sqrt_one_minus_alpha * eps_theta) / sqrt_alpha

    if step < sample_step - 1:
        next_i = new_timesteps[step + 1]
        alpha_next = alpha_cumprod[next_i]

        coef1 = torch.sqrt(alpha_next) * x0_pred
        coef2 = torch.sqrt(1 - alpha_next) * eps_theta
        x = coef1 + coef2
    else:
        x = x0_pred

    print("x")
    print(x.mean())
    print(x.max())
    print(x.min())

x
tensor(-0.0034, device='cuda:0')
tensor(13.4512, device='cuda:0')
tensor(-7.3128, device='cuda:0')
x
tensor(-0.0044, device='cuda:0')
tensor(16.3272, device='cuda:0')
tensor(-7.9543, device='cuda:0')
x
tensor(-0.0071, device='cuda:0')
tensor(15.8093, device='cuda:0')
tensor(-13.8843, device='cuda:0')
x
tensor(-0.0099, device='cuda:0')
tensor(21.7189, device='cuda:0')
tensor(-25.9212, device='cuda:0')
x
tensor(-0.0121, device='cuda:0')
tensor(132.7944, device='cuda:0')
tensor(-104.0345, device='cuda:0')
x
tensor(-0.0158, device='cuda:0')
tensor(772.6113, device='cuda:0')
tensor(-926.1382, device='cuda:0')
x
tensor(-0.0198, device='cuda:0')
tensor(6848.5234, device='cuda:0')
tensor(-5588.8984, device='cuda:0')
x
tensor(-0.0429, device='cuda:0')
tensor(39651.9844, device='cuda:0')
tensor(-47101.6094, device='cuda:0')
x
tensor(-0.1577, device='cuda:0')
tensor(332852.5000, device='cuda:0')
tensor(-266957.6250, device='cuda:0')
x
tensor(-1.5128, device='cuda:0')
tensor(1680839., device='cu

In [33]:
output = F.interpolate(x, scale_factor=ds_factor, mode='bilinear')
print(output.mean())
print(output.max())
print(output.min())
output = remove_padding(output, padding)

tensor(-1.2355e+11, device='cuda:0')
tensor(3.3903e+16, device='cuda:0')
tensor(-3.5084e+16, device='cuda:0')


In [34]:
output = output.squeeze(0).squeeze(0).detach().cpu().numpy()

In [35]:
output_denorm = (output * std_val + mean_val) * max_val
output_denorm += outlier_residual
output_denorm = np.clip(output_denorm, 0, upper_boundary)

In [36]:
output_denorm = output_denorm.astype(str(ds.pixel_array.dtype))
ds.PixelData = output_denorm.tobytes()

In [37]:
ds.save_as("./results/DCM_CHEST_00118.dcm")